In [111]:
import os
import itertools
from functools import reduce

from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import *

from sklearn.metrics import PrecisionRecallDisplay
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler

from sklearn.metrics import classification_report, accuracy_score, make_scorer
from sklearn.model_selection import cross_validate
from sklearn.svm import SVC

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
import plotly.express as px

import numpy
import pandas
import seaborn

from lab_v2.io import read_file

In [112]:
# FILE PATHS
DRAW_T03 = './data/draw/draw-T0.3.jsonl' 
DRAW_T07 = './data/draw/draw-T0.7.jsonl' 
CSQA_T07 = './data/csqa/csqa-T0.7.jsonl' 
LAST_LETTERS_T07 = './data/last_letters/last_letters-T0.7.jsonl' 

ATTRIBUTES = ["majority_distance", "majority_distance_squared", "shannon_entropy", "gini_impurity"]
CLASS = 'majority_correct'
K_FOLDS = 5
RANDOM_STATE = 0
FILE_PATH = CSQA_T07

CACHE = 'cache/csqa/csqa-T0.3.json'
CACHE_ROS = 'cache/csqa/csqa-T0.3-ROS.json'

DATA_EXPLORATION = False

In [113]:
data = read_file(FILE_PATH)
data.head(1)

,majority_distance,majority_distance_squared,shannon_entropy,gini_impurity,majority_correct
0,1.378000e-07,0.0,0.0,0.0,True


In [114]:
if DATA_EXPLORATION:
    fig = px.scatter_3d(data, x='majority_distance', y='shannon_entropy', z='gini_impurity', color=CLASS, 
                        color_discrete_sequence=[px.colors.sequential.Plasma_r[3], px.colors.sequential.Plasma_r[-1]])
    fig.update_traces(marker=dict(size=3), selector=dict(mode='markers'))
    fig.update_traces(marker=dict(opacity=0.75), selector=dict(mode='markers'))
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
    fig.show()

## **Data exploration**

In [115]:
if DATA_EXPLORATION:
    seaborn.set_theme(style='ticks')
    seaborn.pairplot(data, hue="majority_correct", plot_kws={'alpha': 0.4})
    print('')

In [116]:
data_x = data[ATTRIBUTES]
data_y = data[CLASS]

In [117]:
classification_reports = []
def classification_report_scorer(y_true, y_pred):
    classification_reports.append(classification_report(y_true, y_pred))
    return accuracy_score(y_true, y_pred)

In [118]:
def report_average(*args):
    report_list = list()
    for report in args:
        splited = [' '.join(x.split()) for x in report.split('\n\n')]
        header = [x for x in splited[0].split(' ')]
        data = numpy.array(splited[1].split(' ')).reshape(-1, len(header) + 1)
        data = numpy.delete(data, 0, 1).astype(float)
        rest = splited[2].split(' ')
        accuarcy =numpy.array([0, 0, rest[1], rest[2]]).astype(float).reshape(-1, len(header))
        macro_avg = numpy.array([rest[5:9]]).astype(float).reshape(-1, len(header))
        weighted_avg = numpy.array([rest[11:]]).astype(float).reshape(-1, len(header))
        #avg_total = numpy.array([x for x in avg]).astype(float).reshape(-1, len(header))
        df = pandas.DataFrame(numpy.concatenate((data, accuarcy,macro_avg,weighted_avg)), columns=header)
        report_list.append(df)
    res = reduce(lambda x, y: x.add(y, fill_value=0), report_list) / len(report_list)
    return res.rename(index={res.index[-3]: 'accuracy',res.index[-2]: 'macro_avg',res.index[-1]: 'weighted_avg'})

In [119]:
class ReportAverage:
    def __init__(self):
        self.classification_reports = []

    def classification_report_scorer(self, y_true, y_pred):
        self.classification_reports.append(classification_report(y_true, y_pred))
        return accuracy_score(y_true, y_pred)
    
    def average_report(self):
        average = report_average(*self.classification_reports)
        return {
            'precision_True': average.loc[0]['precision'],
            'recall_True': average.loc[0]['recall'],
            'f1-score_True': average.loc[0]['f1-score'],
            'precision_False': average.loc[1]['precision'],
            'recall_False': average.loc[1]['recall'],
            'f1-score_False': average.loc[1]['f1-score'],
            'f1-score_Average': (average.loc[0]['f1-score'] + average.loc[1]['f1-score']) / 2,
            'accuracy': average.loc['accuracy']['f1-score']
        }

In [120]:
def my_cross_validate(model, sampler, data_x, data_y, scoring, cv=5):
    kfold = KFold(n_splits=cv)
    results = []
    for train_idx, test_idx, in kfold.split(data_x):
        X_train, X_test = data_x.iloc[train_idx], data_x.iloc[test_idx]
        y_train, y_test = data_y.iloc[train_idx], data_y.iloc[test_idx]
        
        if sampler != None: X_train, y_train = sampler.fit_resample(X_train, y_train)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        results.append(scoring(y_test, y_pred))
    return sum(results) / len(results)

In [127]:
scorer = ReportAverage()
my_cross_validate(RandomForestClassifier(), RandomOverSampler(), data_x, data_y, scoring=scorer.classification_report_scorer, cv=K_FOLDS)
scorer.average_report()

{'precision_True': 0.72,
 'recall_True': 0.392,
 'f1-score_True': 0.51,
 'precision_False': 0.648,
 'recall_False': 0.8780000000000001,
 'f1-score_False': 0.7460000000000001,
 'f1-score_Average': 0.6280000000000001,
 'accuracy': 0.664}

In [122]:
class MLExploration:
    TEMPLATE_DICT = {
        'model': '', 
        'hyperparameters': '', 
        'precision_True': '', 
        'recall_True': '', 
        'f1-score_True': '', 
        'precision_False': '', 
        'recall_False': '', 
        'f1-score_False': '',
        'f1-score_Average': 0,
        'accuracy': ''
    }
    RANDOM_STATE = 42

    def __init__(self, data_x, data_y, output_file_path, sampler=None):
        self.data_x = data_x
        self.data_y = data_y
        self.output_file_path = output_file_path
        self.sampler = sampler
        self.explored_models = pandas.DataFrame([MLExploration.TEMPLATE_DICT])
        if os.path.exists(output_file_path): 
            self.explored_models = pandas.read_json(output_file_path, orient='split')

    def grid_search(self, model, parameters):
        parameter_combinations = self.__parameter_product(parameters)
        for combination in parameter_combinations:
            print(combination)
            self.explore_model(model, combination)

    def explore_model(self, model, hyperparameters):
        index = MLExploration.hash(model, hyperparameters)
        if index in self.explored_models.index: return self.explored_models.loc[index]

        scorer = ReportAverage()
        my_cross_validate(model(**hyperparameters), self.sampler, data_x, data_y, scoring=scorer.classification_report_scorer)

        self.explored_models.loc[index] = [
            MLExploration.hash_model(model), 
            MLExploration.hash_hyperparameters(hyperparameters),
            *scorer.average_report().values()
        ]
        self.explored_models.to_json(self.output_file_path, orient='split')

    def hash_model(model):
        return str(model.__name__)
    
    def hash_hyperparameters(hyperparameters):
       return str(sorted(hyperparameters.items(), key=lambda x:x[0]))
    
    def hash(model, hyperparameters):
        model = MLExploration.hash_model(model)
        hyperparameters = MLExploration.hash_hyperparameters(hyperparameters)
        return str((model, hyperparameters))
    
    def __parameter_product(self, parameters):
        keys, values = zip(*parameters.items())
        experiments = [dict(zip(keys, v)) for v in itertools.product(*values)]
        return experiments

In [123]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [124]:
def classification_report_scorer(y_true, y_pred):
    classification_report(y_true, y_pred) # print classification report
    return accuracy_score(y_true, y_pred) # return accuracy score

In [125]:
csqa = MLExploration(data_x, data_y, CACHE, RandomOverSampler())

In [126]:
csqa.explore_model(AdaBoostClassifier, {})
csqa.explore_model(RandomForestClassifier, {})
csqa.explore_model(ExtraTreesClassifier, {})
csqa.explore_model(GradientBoostingClassifier, {})
csqa.explore_model(GaussianProcessClassifier, {})
csqa.explore_model(GaussianNB, {})
csqa.explore_model(KNeighborsClassifier, {})
csqa.explore_model(MLPClassifier, {})
csqa.explore_model(SVC, {})
csqa.explore_model(DecisionTreeClassifier, {})
csqa.explored_models.sort_values(by='f1-score_Average').style.hide(axis='index')

TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

In [ ]:
ros = RandomOverSampler(random_state=RANDOM_STATE)
ros = ADASYN(random_state=RANDOM_STATE)
ros = SMOTE(random_state=RANDOM_STATE)
data_x_resampled, data_y_resampled = ros.fit_resample(data_x, data_y)

In [ ]:
csqa_ros = MLExploration(data_x_resampled, data_y_resampled, CACHE_ROS)

In [ ]:
csqa_ros.explore_model(AdaBoostClassifier, {})
csqa_ros.explore_model(RandomForestClassifier, {})
csqa_ros.explore_model(ExtraTreesClassifier, {})
csqa_ros.explore_model(GradientBoostingClassifier, {})
csqa_ros.explore_model(GaussianProcessClassifier, {})
csqa_ros.explore_model(GaussianNB, {})
csqa_ros.explore_model(KNeighborsClassifier, {})
csqa_ros.explore_model(MLPClassifier, {})
csqa_ros.explore_model(SVC, {})
csqa_ros.explore_model(DecisionTreeClassifier, {})
csqa_ros.explored_models.sort_values(by='f1-score_Average').style.hide(axis='index')

model,hyperparameters,precision_True,recall_True,f1-score_True,precision_False,recall_False,f1-score_False,f1-score_Average,accuracy
,,,,,,,,0.000000,
AdaBoostClassifier,[],0.456000,0.276000,0.338000,0.842000,0.920000,0.878000,0.608000,0.796000
DecisionTreeClassifier,[],0.712000,0.368000,0.482000,0.638000,0.880000,0.742000,0.612000,0.654000
KNeighborsClassifier,[],0.612000,0.486000,0.540000,0.650000,0.758000,0.698000,0.619000,0.634000
RandomForestClassifier,[],0.714000,0.386000,0.502000,0.644000,0.876000,0.744000,0.623000,0.662000
GaussianNB,[],0.750000,0.386000,0.510000,0.650000,0.898000,0.752000,0.631000,0.672000
SVC,[],0.758000,0.384000,0.510000,0.648000,0.904000,0.756000,0.633000,0.674000
MLPClassifier,[],0.746000,0.396000,0.516000,0.652000,0.894000,0.754000,0.635000,0.672000
ExtraTreesClassifier,[],0.432000,0.418000,0.428000,0.862000,0.868000,0.866000,0.647000,0.782000
GradientBoostingClassifier,[],0.530000,0.340000,0.416000,0.858000,0.928000,0.890000,0.653000,0.814000


In [ ]:
def my_cross_validate(data_x, data_y, model, sampler, scoring, k_folds=5):
    cv = StratifiedKFold(n_splits=k_folds)
    results = []
    for train_idx, test_idx, in cv.split(data_x, data_y):
        X_train, y_train = data_x[train_idx], data_y[train_idx]
        X_test, y_test = data_x[test_idx], data_y[test_idx]
        X_train, y_train = sampler.fit_sample(X_train, y_train)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        results.append(scoring(y_test, y_pred))
    return sum(results) / len(results)